# Example usage of the diferent models

In [ ]:
import numpy as np
import os
import sys

cwd = os.getcwd()
base_dir = os.path.dirname(cwd)
src_dir = base_dir + "/src"
nn_params_dir = cwd + "/training/params"
gbt_params_dir = cwd + "/training/trees"
mpem_dir = base_dir + "/mpem/optimized_parameters"
mpem_file_path = mpem_dir + "/optimized_dipole_5.yaml"
sys.path.insert(0, src_dir)

from calibration import MPEM, MPEM_AVAILABLE, ActuationNet, PotentialNet, DirectNet, DirectGBT

## Included core models

We consider the following models:

- MPEM: model-based baseline,
- ActuationNet: proposed structured learning-based method that predicts an affine field map from position,
$\mathcal{A}_{b}, \mathbf{b}0 = \mathrm{net}(\mathbf{p})$, and via autodiff also provides $\mathcal{A}{g}, \mathbf{g}_0$, such that:
$$
\mathbf{b} = \mathcal{A}_b \mathbf{i} + \mathbf{b}_0,\quad \mathbf{g} = \mathcal{A}_g \mathbf{i} + \mathbf{g}_0
\tag{1}
$$
- PotentialNet: proposed structured learning-based method that predicts scalar potentials $\Phi, \phi_0=\mathrm{net}(\mathbf{p})$ and defines the affine maps through derivatives:
$\mathcal{A}_b=\nabla\Phi$, $\mathbf{b}0=\nabla\phi_0$. The gradient maps $\mathcal{A}{g}, \mathbf{g}_0$ follow from taking the Hessian of the potentials, yielding (1).
- DirectNet: black-box neural net baseline: $\mathbf{b}=net(\mathbf{p}, \mathbf{i}),\quad \mathbf{g}=\nabla\mathbf{b}$ 
- DirectGBT: black-box gradient-boosted linear-tree regressor: $\mathbf{b}=net(\mathbf{p}, \mathbf{i}),\quad \mathbf{g}=\nabla\mathbf{b}$ 


You can load them as:

In [ ]:
# Load models
mpem = MPEM(mpem_file_path) if MPEM_AVAILABLE else None
actuation_net = ActuationNet.load_from(nn_params_dir + "/ActuationNet_100_512x512x512.pt")
potential_net = PotentialNet.load_from(nn_params_dir + "/PotentialNet_100_512x512x512.pt")
direct_net = DirectNet.load_from(nn_params_dir + "/DirectNet_100_512x512x512.pt")
direct_gbt = DirectGBT.load_from(gbt_params_dir + "/DirectGBT_100_128.gbt.zip")

The naming convention for the learning-based models followed in this project is:
- For learning-based models: "ModelName_DatasetPercentage_Structure",
- For MPEM models: "optimized_ModelOrder_DatasetPercentage".

Models all follow the same interface defined in /src/calibration/calibration.py, let's use the ActuationNet for now.

In [ ]:
model = actuation_net

## Field and gradients interface

All models allow for computing the field as a function of the position and currents:
- model.get_field(pos, currents),

as well as grad5 and grad9:
- model.get_grad5(pos, currents)
- model.get_grad9(pos, currents)

Currently, models work with milli-Tesla, meters, and amperes

In [ ]:
currents = np.array([2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) 
pos = np.array([0.01, 0.01, 0.0])

In [ ]:
print("Model Field Outputs at pos", pos, "with currents", currents, "\n")

field = model.get_field(pos, currents)

print(model.name, field)

In [ ]:
print("Model Grad5 Outputs at pos", pos, "with currents", currents, "\n")

grad5 = model.get_grad5(pos, currents)

print(model.name, grad5)

## Currents -> Fields/Gradients

Although we also accommodate a linear interface, we focus on the introduced affine structure with the following interface:

- model.currents_field_jacobian_and_bias(pos) -> (3, num_coils), (3,)
- model.currents_grad5_jacobian_and_bias(pos) -> (5, num_coils), (5,)
- model.currents_full_jacobian_and_bias(pos)  -> (8, num_coils), (8,)

Note that the black-box/direct models do not implement the methods for the affine interface.

In [ ]:
print("Model Field Actuation matrix at pos", pos, "\n")

A_field, b0 = model.currents_field_jacobian_and_bias(pos)

print(model.name, "A_field: ", A_field, "\nb0: ", b0)

In [ ]:
print("Model Grad5 Actuation matrix at pos", pos, "\n")

A_grad, g0 = model.currents_grad5_jacobian_and_bias(pos)

print(model.name, "A_grad: ", A_grad, "\ng0: ", g0)

In [ ]:
print("Model Full Actuation matrix at pos", pos, "\n")

A_full, bg0 = model.currents_full_jacobian_and_bias(pos)

print(model.name, "A_full: ", A_full, "\nbg0: ", bg0)

## Inverse problem

The affine models implement a method for computing the inverse problem:
- model.get_currents(pos, target_field, target_grad) -> (num_coils,)

This is done via the pseudo inverse: $\mathbf{i} = \mathcal{A}^\dagger (\{\mathbf{b}_{target}, \mathbf{g}_{target}\}-\{\mathbf{b}_0, \mathbf{g}_0\})$

In the .get_currents method, either the target_field or the target_grad5 can be passed as None (the default). In this case, the corresponding actuation matrix will not be considered.

In [ ]:
target_field = np.array([3.0, 2.0, 1.0])
target_grad5 = np.array([50, 100, 250, 46, 35])
print("Model Currents for achieving field", target_field, "and grad5", target_grad5, "at pos", pos, "\n")

currents = model.get_currents(pos, target_field, target_grad5)

print(model.name, "Currents: ", currents)